In [2]:
import json
import urllib
import numpy as np
import re
import tqdm
import os
import sys
import cv2
#if cairopath not in sys.path:
#    sys.path.append(cairopath)
import cairosvg

In [3]:
#data=json.load(open('../3rd/emoji-kitchen/scripts/emojiOutput.json'))
#url='https://emojikitchen.dev/assets/index-39d501e3.js'
#urllib.request.urlretrieve(url, '../data/emojisite.json')
rawdata=open('../data/emojisite.json').read()
#modified_json_str = add_quotes_to_ints_and_words(rawdata)
#find the patterns
pattern=r'"https://www\.gstatic\.com/android/keyboard/emojikitchen/2[^"]+"'
#data=json.loads(modified_json_str)   
alllinks=re.findall(pattern,rawdata)
alllinks.sort()
onlynew={}
for link in alllinks:
    name=link.split('/')[-1][:-1]
    onlynew[name]=link[1:-1]

In [4]:
todownload=set()
for name in onlynew.keys():
    todownload.add(name.split('.')[0].split('_')[0])
    todownload.add(name.split('.')[0].split('_')[1])
todownload=list(todownload)

In [6]:
os.mkdir('../data/emojis')
os.mkdir('../data/emojis/bare')
os.mkdir('../data/emojis/barepng')
os.mkdir('../data/emojis/labeladbares')
os.mkdir('../data/emojis/blended')

In [7]:
ctr=0
for name in tqdm.tqdm(todownload):
    url=f'https://raw.githubusercontent.com/googlefonts/noto-emoji/main/svg/emoji_{name}.svg'
    try:
        urllib.request.urlretrieve(url, f"../data/emojis/bare/{name}.svg")
    except:
        ctr=ctr+1
        if ctr%10==0:
            print("failed ", url)

 11%|█         | 71/638 [00:16<02:02,  4.64it/s]

failed  https://raw.githubusercontent.com/googlefonts/noto-emoji/main/svg/emoji_u1f336-ufe0f.svg


 23%|██▎       | 148/638 [00:33<01:59,  4.10it/s]

failed  https://raw.githubusercontent.com/googlefonts/noto-emoji/main/svg/emoji_u1f5d1-ufe0f.svg


 32%|███▏      | 201/638 [00:45<01:40,  4.35it/s]

...

 79%|███████▉  | 507/638 [01:56<00:32,  4.08it/s]

failed  https://raw.githubusercontent.com/googlefonts/noto-emoji/main/svg/emoji_u1f3dd-ufe0f.svg


 97%|█████████▋| 620/638 [02:22<00:04,  4.42it/s]

failed  https://raw.githubusercontent.com/googlefonts/noto-emoji/main/svg/emoji_u2708-ufe0f.svg


100%|██████████| 638/638 [02:26<00:00,  4.34it/s]


In [8]:
len(os.listdir('../data/emojis/bare/'))

555

In [9]:
#cairopath = 'C:\GTK\bin'
root="../data/emojis/bare/"
dest="../data/emojis/barepng/"
# Convert SVG to PNG
for file in tqdm.tqdm(os.listdir(root)):
    destpath=dest+file.split('.')[0]+'.png'
    cairosvg.svg2png(url=root+file, write_to=destpath,output_width=534,output_height=534)
    # REMOVE ALPHA
    bgra=cv2.imread(destpath,cv2.IMREAD_UNCHANGED)
    if bgra.shape[-1]>2:
        alpha=bgra[...,-1:]>127
        bgr=bgra[...,:-1]*alpha+255*(1-alpha)
        bgr=bgr.astype(np.uint8)
    else:
        bgr=bgra
    #print(bgr.shape)
    cv2.imwrite(destpath,bgr)

100%|██████████| 555/555 [00:20<00:00, 27.67it/s]


In [10]:
for name, url in tqdm.tqdm(onlynew.items()):
    try:
        urllib.request.urlretrieve(url, f"../data/emojis/blended/{name}")
    except:
        print("failed ", url)

  4%|▍         | 1560/40296 [02:42<1:06:02,  9.78it/s]

In [45]:
xmldatapath='../data/emojidescript.xml'
xmldata=open(xmldatapath).read()
pattern=r'alt="([^"]+)"\s+src="([^"]+\.svg)"'
emojidict={k.split('_')[-1].split('/')[-1].split('.')[0]:v for (v,k) in re.findall(pattern,xmldata)}

In [54]:
with open('emojidict.json','w') as f:
    json.dump(emojidict,f)

In [50]:
for file in tqdm.tqdm(os.listdir(root)):
    destpath=dest+file.split('.')[0]+'.png'
    name=destpath.split('/')[-1].split('.')[0]
    cairosvg.svg2png(url=root+file, write_to=destpath,output_width=534,output_height=534)
    # REMOVE ALPHA
    bgra=cv2.imread(destpath,cv2.IMREAD_UNCHANGED)
    if bgra.shape[-1]>2:
        alpha=bgra[...,-1:]>127
        bgr=bgra[...,:-1]*alpha+255*(1-alpha)
        bgr=bgr.astype(np.uint8)
    else:
        bgr=bgra
    cv2.imwrite('../data/emojis/labeladbares/'+emojidict[name]+'.png',bgr)

100%|██████████| 555/555 [00:26<00:00, 20.79it/s]
